## TOOL UPDATED: October 20 2023, Henrik Loecke

##Double click this cell to see full description.

##You must restart the kernel after updating Calibration_Variables.py!

<!-- 

To run this notebook, click menu Cell -> Run All

User input has been moved away from this notebook so it can easily be replaced by new versions.
 
Please open Calibration_Variables.py, in the same folder as this notebook, to edit user input there.

All variables with path must start with 'r', e.g. r'C:\Projects'

It must contain the following variables:

model_area:                          Short area name like 'VSA' or LISA'
generate_confidence_csvs:            Generate csv files for confidence maps, True/False
result_specs_csv:                    CSV file linking network and runoff result file. Only needed if runoff imported.
map_point_spacing:                   Space between dots in confidence maps, e.g. 100 (in meter but number withouth unit)
use_accumulation:                    Use for models with proper tree structure, all models except VSA, True/False
slope_source_unit_meter_per_meter    This is the case for NSSA and FSA, in VSA it is per thousand
model_area_strict_match:             If True, accept 'VSA' but not 'VSA-2019'. If False, accept both.
output_folder:                       Folder path where reports are to be created.
result_folder:                       Folder path of result files.
calibration_sheet:                   Full path, folder included, of calibration parameter sheet.
model:                               Full path, folder included, of model database.
summation_csv:                       Full path, folder included, of summation.csv.
node_csv:                            Full path, folder included, of MH_Zones.csv.
outfall_csv:                         Full path, folder included, of Outfall_Summary.csv.
rainfall_dfs0_file:                  Full path, folder included, of rainfall dfs0 file.
map_folder:                          Folder path where report maps are located.
dfs0_folders:                        Python list of all folders holding measurement dfs0 files.
 -->


In [2]:
#PERMANENT CELL 1

import os
import re
import mikeio
import mikeio1d
from mikeio1d.res1d import Res1D
from mikeio.dfs0 import Dfs0
import numpy as np
import pandas as pd
import datetime as dt
import pickle
import plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from IPython.display import clear_output
import math
import sqlite3
from Calibration_Variables import *
import copy
import ctypes
import traceback
MessageBox = ctypes.windll.user32.MessageBoxA


In [3]:
#PERMANENT CELL 1
#Import dfs0 files

try:

    #Import rain gauges
    rainfall = mikeio.read(rainfall_dfs0_file).to_dataframe()
    warnings = []

    #Import flow/level gauges
    gauge_ids = []
    first_dfs0 = True
    for dfs0_folder in dfs0_folders:
        for f in os.listdir(dfs0_folder):
            if f[-5:]=='.dfs0':


                if 'ps' in f.lower() and f[:2].lower() != 'ps':
                    gauge_id = re.split(r'[.]',f)[0]
                else:
                    gauge_id = re.split(r'_|[.]',f)[0]
                res = mikeio.read(dfs0_folder + '\\' + f)
                ts = res.to_dataframe()

                gauge_ids.append(gauge_id)

                first_level = True
                second_level = True
                first_velocity = True
                for i, column in enumerate(ts.columns):
                    if i == 0:
                        ts.rename(columns={ts.columns[0]:'Flow'},inplace=True)
                        if 'meter pow 3 per sec' in str(res.items[0]):
                            ts.Flow = ts.Flow * 1000
                        elif 'liter per sec' in str(res.items[0]):
                            ts.Flow = ts.Flow 
                        else:
                            warnings.append('First item in ' + f + ' does not appear to be type Discharge. This is not imported.')
                            ts.Flow = np.nan

                    elif str(res.items[i])[-21:] == '<Water Level> (meter)': 
                        if first_level == True:
                            ts.rename(columns={ts.columns[i]:'Level'},inplace=True)
                            first_level = False
                        elif second_level == True:
                            ts.rename(columns={ts.columns[i]:'Level2'},inplace=True)
                            second_level = False


                    elif str(res.items[i])[-31:] == '<Flow velocity> (meter per sec)' and first_velocity == True:
                        ts.rename(columns={ts.columns[i]:'Velocity'},inplace=True)
                        first_velocity = False

                if not 'Flow' in ts.columns:
                    ts['Flow'] = np.nan                     
                if not 'Velocity' in ts.columns:
                    ts['Velocity'] = np.nan
                if not 'Level' in ts.columns:
                    ts['Level'] = np.nan
                if not 'Level2' in ts.columns:
                    ts['Level2'] = np.nan

                ts['Gauge'] = gauge_id
                ts = ts[['Gauge','Flow','Level','Level2','Velocity']]
                ts['Seconds'] = ts.index.to_series().diff().astype('timedelta64[s]').fillna(method='bfill')
                ts['Volume'] = ts.Flow * ts.Seconds / 1000
                if first_dfs0 == True:
                    measured = ts.copy()
                else:
                    measured = pd.concat([measured,ts])
                first_dfs0 = False

    for warning in warnings:
        print (warning)

except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 5', b'Error', 0)
    raise ValueError("Error")

In [6]:
measured.columns

Index(['Gauge', 'Flow', 'Level', 'Level2', 'Velocity', 'Seconds', 'Volume'], dtype='object')

In [4]:
rainfall

,Rainfall_BU07,Rainfall_BU29,Rainfall_BU70,Rainfall_BU80,Rainfall_CW09,Rainfall_DM44,Rainfall_DT34,Rainfall_LN46,Rainfall_PQ38,Rainfall_PT11,Rainfall_QT10,Rainfall_QT77,Rainfall_QT87,Rainfall_SU42,Rainfall_SU48,Rainfall_SU56,Rainfall_WK47
2016-10-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-10-01 00:05:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-10-01 00:10:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-10-01 00:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-10-01 00:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-31 23:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-31 23:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-31 23:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-31 23:55:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
# Pivot the DataFrame to create the new structure
flow_df = measured.pivot(columns='Gauge', values='Flow')

# Reindex the new DataFrame with the original datetime index
# new_df.index = measured.index
# flow_df.to_dfs0('Flow.dfs0',ItemInfo('Discharge',EUMType.Discharge, EUMUnit.literpersecond))
flow_df /= 1000
flow_df.to_dfs0(r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\Calibration_2022\WWF_Calibration_Reports\DASH\measuredflow.dfs0")

In [33]:
flow_df

Gauge,20th_Street_PS,AL1,AL3,AN1,BI1,BIN18,BN21,BNC9,BNP17,CQ1,...,Port_Moody_PS,Royal_Avenue_PS,SAS9A,SYC32,SYC44,SYC9,Sapperton_PS,Short_Street_PS,Sperling_PS,Westridge_1_PS
2021-01-01 00:00:00,0.003992,NaN,NaN,10.19590,0.000000,NaN,NaN,NaN,NaN,0.347157,...,0.313258,0.145778,NaN,NaN,NaN,NaN,2.10371,0.0,0.345550,0.014319
2021-01-01 00:05:00,0.003707,NaN,NaN,10.41680,0.000000,NaN,NaN,NaN,NaN,0.345887,...,0.313985,0.146531,NaN,NaN,NaN,NaN,2.10784,0.0,0.343724,0.071997
2021-01-01 00:10:00,0.000000,NaN,NaN,10.22670,0.000000,NaN,NaN,NaN,NaN,0.342025,...,0.310305,0.147634,NaN,NaN,NaN,NaN,2.12478,0.0,0.346599,0.062244
2021-01-01 00:15:00,0.000000,NaN,NaN,9.84897,0.000000,NaN,NaN,NaN,NaN,0.342775,...,0.313223,0.148007,NaN,NaN,NaN,NaN,2.13178,0.0,0.344631,0.029697
2021-01-01 00:20:00,0.000000,NaN,NaN,9.65628,0.000000,NaN,NaN,NaN,NaN,0.347550,...,0.311215,0.148210,NaN,NaN,NaN,NaN,2.10607,0.0,0.344325,0.093855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-17 23:35:00,0.000000,NaN,NaN,6.06965,0.676383,NaN,NaN,NaN,NaN,0.023777,...,0.000000,0.000000,NaN,NaN,NaN,NaN,1.71678,0.0,0.203311,0.001083
2022-10-17 23:40:00,0.006463,NaN,NaN,6.02812,0.689462,NaN,NaN,NaN,NaN,0.029844,...,0.000000,0.000016,NaN,NaN,NaN,NaN,1.68703,0.0,0.197647,0.000000
2022-10-17 23:45:00,0.000010,NaN,NaN,6.16589,0.679926,NaN,NaN,NaN,NaN,0.023436,...,0.000000,0.000000,NaN,NaN,NaN,NaN,1.68843,0.0,0.193270,0.000000
2022-10-17 23:50:00,0.000000,NaN,NaN,6.28945,0.678687,NaN,NaN,NaN,NaN,0.128495,...,0.307639,0.000016,NaN,NaN,NaN,NaN,1.72388,0.0,0.191082,0.000000


In [29]:
# Pivot the DataFrame to create the new structure
level_df = measured.pivot(columns='Gauge', values='Level')

# Reindex the new DataFrame with the original datetime index
# new_df.index = measured.index
# flow_df.to_dfs0('Flow.dfs0',ItemInfo('Discharge',EUMType.Discharge, EUMUnit.literpersecond))
level_df.to_dfs0('Level.dfs0')

In [31]:
level_df

Gauge,20th_Street_PS,AL1,AL3,AN1,BI1,BIN18,BN21,BNC9,BNP17,CQ1,...,Port_Moody_PS,Royal_Avenue_PS,SAS9A,SYC32,SYC44,SYC9,Sapperton_PS,Short_Street_PS,Sperling_PS,Westridge_1_PS
2021-01-01 00:00:00,5.70589,NaN,NaN,NaN,4.67150,NaN,NaN,NaN,NaN,28.520000,...,1.15550,-0.206343,NaN,NaN,NaN,NaN,0.340648,-0.708783,12.5069,26.890200
2021-01-01 00:05:00,5.40517,NaN,NaN,NaN,4.67380,NaN,NaN,NaN,NaN,28.520000,...,1.11125,-0.151845,NaN,NaN,NaN,NaN,0.429189,-0.633750,12.5069,27.226900
2021-01-01 00:10:00,5.44064,NaN,NaN,NaN,4.67483,NaN,NaN,NaN,NaN,28.515301,...,1.07600,-0.094843,NaN,NaN,NaN,NaN,0.483250,-0.609922,12.5069,26.686300
2021-01-01 00:15:00,5.52193,NaN,NaN,NaN,4.67384,NaN,NaN,NaN,NaN,28.520000,...,1.04433,-0.053157,NaN,NaN,NaN,NaN,0.542313,-0.555625,12.5069,27.256201
2021-01-01 00:20:00,5.60781,NaN,NaN,NaN,4.67744,NaN,NaN,NaN,NaN,28.520000,...,1.00933,-0.032469,NaN,NaN,NaN,NaN,0.580875,-0.547422,12.5069,26.820900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-17 23:35:00,5.75405,NaN,NaN,NaN,4.56984,NaN,NaN,NaN,NaN,28.220301,...,1.72904,-0.794127,NaN,NaN,NaN,NaN,-0.225780,-0.308750,11.8890,26.538500
2022-10-17 23:40:00,5.64748,NaN,NaN,NaN,4.56822,NaN,NaN,NaN,NaN,28.228100,...,1.78371,-0.557803,NaN,NaN,NaN,NaN,-0.294312,-0.257383,11.8716,26.799700
2022-10-17 23:45:00,5.35309,NaN,NaN,NaN,4.56905,NaN,NaN,NaN,NaN,28.219999,...,1.83704,-0.835525,NaN,NaN,NaN,NaN,-0.294312,-0.187070,11.8596,27.036200
2022-10-17 23:50:00,5.35562,NaN,NaN,NaN,4.56450,NaN,NaN,NaN,NaN,28.310200,...,1.58726,-1.079680,NaN,NaN,NaN,NaN,-0.294312,-0.111550,11.8407,27.181700


In [15]:
help(flow_df.to_dfs0, )

Help on method dataframe_to_dfs0 in module mikeio.dfs0:

dataframe_to_dfs0(filename, itemtype=None, unit=None, items=None, title=None, dtype=None) method of pandas.core.frame.DataFrame instance
    Create a dfs0
    
    Parameters
    ----------
    filename: str
        filename to write output
    itemtype: EUMType, optional
        Same type for all items
    unit: EUMUnit, optional
        Same unit for all items
    items: list[ItemInfo]
        Different types, units for each items, similar to `create`
    title: str, optional
        Title of dfs0 file
    dtype : np.dtype, optional
            default np.float32

